In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from collections import OrderedDict
from itertools import combinations
import operator
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

In [2]:
# import data
housing = pd.read_csv('https://raw.githubusercontent.com/DataSciencePros/data_science_workshop/master/app/data/housing_processed.csv')
housing.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,...,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_0,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,1,60,65.0,8450,7,5,2003,2003,196.0,Gd,TA,Gd,TA,No,GLQ,706,Unf,0,150,856,Ex,Y,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,RFn,2,548,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2,20,80.0,9600,6,8,1976,1976,0.0,TA,TA,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,Ex,Y,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,RFn,2,460,...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,3,60,68.0,11250,7,5,2001,2002,162.0,Gd,TA,Gd,TA,Mn,GLQ,486,Unf,0,434,920,Ex,Y,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,RFn,2,608,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,4,70,60.0,9550,7,5,1915,1970,0.0,TA,TA,TA,Gd,No,ALQ,216,Unf,0,540,756,Gd,Y,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Unf,3,642,...,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,5,60,84.0,14260,8,5,2000,2000,350.0,Gd,TA,Gd,TA,Av,GLQ,655,Unf,0,490,1145,Ex,Y,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,RFn,3,836,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [3]:
# data setup
housing_ml = housing.drop(columns=["ExterQual"])
housing_ml = housing.loc[:,housing.dtypes != object]
#housing_ml = housing_ml.set_index('Id')
housing_ml.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,ExterQual_Coded,ExterCond_Coded,BsmtQual_Coded,...,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_0,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2,548,0,61,0,0,0,0,0,2,2008,208500,3,2,4,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,2,460,298,0,0,0,0,0,0,5,2007,181500,2,2,4,...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2,608,0,42,0,0,0,0,0,9,2008,223500,3,2,4,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,3,642,0,35,272,0,0,0,0,2,2006,140000,2,2,3,...,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,3,836,192,84,0,0,0,0,0,12,2008,250000,3,2,4,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [4]:
sale_price = housing_ml.pop('SalePrice')

1. Split the data for training and testing, to use 80 percent as training data. Use 21 as your randomization seed.

In [5]:
# split data - random_state = 21, train_size = .8
X = housing_ml.values
y = sale_price.values 

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 21, train_size = .8)
print("X_train:", X_train.shape, "y_train:", y_train.shape, "X_test:", X_test.shape, "y_test:", y_test.shape)

X_train: (1168, 221) y_train: (1168,) X_test: (292, 221) y_test: (292,)


2. Analyze which feature alone would give the best prediction, list the scores and RMSE errors achieved by the top 10 predictors by score.

   (create a loop to train a linear model for each feature and save the scores and RMSE and sort by score)

In [6]:
# linear regression model for each feature
features = housing_ml.columns.to_list()
results = {}

linear_model = LinearRegression()

for i in range(len(features)):
    train_x = X_train[:,i].reshape(-1,1)
    test_x = X_test[:,i].reshape(-1,1)

    model = linear_model.fit(train_x, y_train)

    y_pred = model.predict(test_x)

    score = linear_model.score(test_x, y_test)
    rmse = mean_squared_error(y_test, y_pred, squared = False)

    results[i] = [features[i], score, rmse]
  
sorted_results = sorted(results.values(), key=operator.itemgetter(1), reverse=True)[:10]
sorted_results

[['OverallQual', 0.6454631197278842, 49018.435788124996],
 ['ExterQual_Coded', 0.4991753471267155, 58260.148655350655],
 ['GrLivArea', 0.44428966678140597, 61369.55515927838],
 ['KitchenQual_Coded', 0.43970293723342646, 61622.30113827634],
 ['TotalBsmtSF', 0.42491266111423953, 62430.33167581401],
 ['1stFlrSF', 0.4166679048480356, 62876.2562386846],
 ['GarageCars', 0.41193116659665824, 63131.02210090164],
 ['GarageArea', 0.4073263621599067, 63377.71018105261],
 ['BsmtQual_Coded', 0.3409700148262619, 66831.5181312807],
 ['GarageFinish_Coded', 0.31628495442245047, 68071.65468394081]]

3. Select all possible 2 pairs of these top 10 predictors, and train 45 linear models, list the scores and RMSE errors achieved by the top 10 predictors by score.

In [7]:
# create combinations of the top 10 features
top_columns = [item[0] for item in sorted_results]
top_features = housing_ml[top_columns]

X_train_10, X_test_10, y_train_10, y_test_10 = train_test_split(top_features, y, random_state = 21, train_size = .8)

feature_combo = list(list(combinations(top_features.columns, 2)))
print(feature_combo)
print(len(feature_combo))

[('OverallQual', 'ExterQual_Coded'), ('OverallQual', 'GrLivArea'), ('OverallQual', 'KitchenQual_Coded'), ('OverallQual', 'TotalBsmtSF'), ('OverallQual', '1stFlrSF'), ('OverallQual', 'GarageCars'), ('OverallQual', 'GarageArea'), ('OverallQual', 'BsmtQual_Coded'), ('OverallQual', 'GarageFinish_Coded'), ('ExterQual_Coded', 'GrLivArea'), ('ExterQual_Coded', 'KitchenQual_Coded'), ('ExterQual_Coded', 'TotalBsmtSF'), ('ExterQual_Coded', '1stFlrSF'), ('ExterQual_Coded', 'GarageCars'), ('ExterQual_Coded', 'GarageArea'), ('ExterQual_Coded', 'BsmtQual_Coded'), ('ExterQual_Coded', 'GarageFinish_Coded'), ('GrLivArea', 'KitchenQual_Coded'), ('GrLivArea', 'TotalBsmtSF'), ('GrLivArea', '1stFlrSF'), ('GrLivArea', 'GarageCars'), ('GrLivArea', 'GarageArea'), ('GrLivArea', 'BsmtQual_Coded'), ('GrLivArea', 'GarageFinish_Coded'), ('KitchenQual_Coded', 'TotalBsmtSF'), ('KitchenQual_Coded', '1stFlrSF'), ('KitchenQual_Coded', 'GarageCars'), ('KitchenQual_Coded', 'GarageArea'), ('KitchenQual_Coded', 'BsmtQual_C

In [8]:
# train 45 linear models
pair_rmses = []
pair_scores = []

for pair in feature_combo:
    pair_train = X_train_10[[pair[0],pair[1]]]
    pair_test = X_test_10[[pair[0],pair[1]]]

    linear_model.fit(pair_train, y_train_10)
    y_pred = linear_model.predict(pair_test)

    pair_rmse = mean_squared_error(y_test_10, y_pred, squared=False)
    pair_rmses.append(pair_rmse)
    pair_score = linear_model.score(pair_test, y_test_10)
    pair_scores.append(pair_score)

In [9]:
# display top 10 predictor pairs 
vals = zip(pair_scores, pair_rmses)
pair_results = dict(zip(feature_combo, vals))

top_pairs = sorted(pair_results.items(), key=operator.itemgetter(1), reverse=True)[:10]

top_pairs

[(('OverallQual', '1stFlrSF'), (0.7173882583039418, 43764.68831784155)),
 (('OverallQual', 'TotalBsmtSF'), (0.706791108152437, 44577.66409611639)),
 (('OverallQual', 'GrLivArea'), (0.6921986779040228, 45673.46700931651)),
 (('OverallQual', 'GarageArea'), (0.6813946935673894, 46468.13490082075)),
 (('OverallQual', 'KitchenQual_Coded'),
  (0.673086953607461, 47070.072304423644)),
 (('OverallQual', 'GarageCars'), (0.6723247870365773, 47124.910051666164)),
 (('OverallQual', 'ExterQual_Coded'), (0.6680147343521529, 47433.82463056837)),
 (('OverallQual', 'BsmtQual_Coded'), (0.6566660974647345, 48237.754631121876)),
 (('OverallQual', 'GarageFinish_Coded'),
  (0.6519020465859777, 48571.27166300008)),
 (('ExterQual_Coded', '1stFlrSF'), (0.6364054863411375, 49640.643170742085))]

4. Train a single model using all features. Calculate RMSE and score. Observe how much of the prediction power was in the 2 pairs, vs all features.

In [10]:
# linear regression model with all features
allFeatures = linear_model.fit(X_train, y_train)
y_pred = allFeatures.predict(X_test)

allRMSE = mean_squared_error(y_test, y_pred, squared = False)
allScore = linear_model.score(X_test, y_test)
print("RMSE for all features:", allRMSE)
print("Score for all features:", allScore)

RMSE for all features: 35623.30345000569
Score for all features: 0.8127547098528082


5. Use the 5NN and 10NN regressor with all features, and list the RMSE and score for these 2 models 

In [11]:
# 5NN regressor with all features
all_5nn = KNeighborsRegressor(n_neighbors = 5)
all_5nn.fit(X_train, y_train)
y_pred_5nn = all_5nn.predict(X_test)

RMSE_5nn = mean_squared_error(y_test, y_pred_5nn, squared = False)
score_5nn = all_5nn.score(X_test, y_test)

print("RMSE for all features:", RMSE_5nn)
print("Score for all features:", score_5nn)

RMSE for all features: 51966.07983368139
Score for all features: 0.6015421069563244


In [12]:
# 10NN regressor with all features
all_10nn = KNeighborsRegressor(n_neighbors = 10)
all_10nn.fit(X_train, y_train)
y_pred_10nn = all_10nn.predict(X_test)

RMSE_10nn = mean_squared_error(y_test, y_pred_10nn, squared = False)
score_10nn = all_10nn.score(X_test, y_test)

print("RMSE for all features:", RMSE_10nn)
print("Score for all features:", score_10nn)

RMSE for all features: 52392.16317256475
Score for all features: 0.5949812006956909


#### Observe if the results are better than linear regression?
The KNN regressors did worse than linear regression. Score for all features linear regression was .81, whereas KNN regressor scores were around .60. 

6) Which regressor is better for inference? linear regression or KNN?

Linear regression is better for inference because it received higher score and lower RMSE than KNN regressor did.